<a href="https://colab.research.google.com/github/2caves/LinearRegressionFromScratch/blob/main/LinearRegressionModelScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression Model for predicting house prices - from scratch!

The purpose of this exercise is to learn about the fundamental inner-workings of a simple linear regression model, without the use of any ML libraries like scikit-learn, keras or pytorch.

As this is the simplest of regression models, we will initially only be considering one feature to predict our house prices - area. Later, we will include more features from the dataset in a vectorized approach to improve the accuracy of our model predictions and further understand how to work with increasing numbers of features.

We'll use stochastic gradient descent as our optimization algorithm, alongside regularization to prevent overfitting on the training data and we'll split our dataset into test and validation sets to measure how our model comparatively performs on examples it hasn't seen before.

If you want to follow along, you can use this as a guide and try your own implementation separately - then compare!


So what's the plan?

- Set up our environment with libraries we'll need
- Grab our dataset we will use to train the model
- Clean up the data to be useful + split into train/val/test sets
- Define our model structure by initialization
- Define how to train the model (predict -> loss -> gradients -> update params)
- Plot our loss curves
- Test our model on the test set

Let's begin from scratch by importing our libraries - numpy (for efficient numeric operations), pandas (for data preprocessing) and matplotlib (for visualization)

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt

Next let's get our dataset from Kaggle. We'll be using House Sales in King County - USA and initially only be looking at the square foot area for each example.

Download the dataset here, unzip *'archive.zip'* and place the .csv contained in your *'LinearRegressionModel/datasets/'* folder.

# Loading dataset via Google Drive:

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# Replace the 'None' for FOLDERNAME and enter the foldername in your Drive where you have saved the unzipped
# dataset csv file, e.g. 'LinearRegressionModel/datasets/'
FOLDERNAME = None

assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

data = pd.read_csv('./kc_house_data.csv')
print(f'This is a portion of our dataset:\n{data.iloc[:4,:6]}')

# Alternatively, load directly from file:
(Be sure to upload the dataset .csv to Colab and put it in a *datasets* folder):

In [ ]:
import sys

sys.path.append('./datasets/')
data = pd.read_csv('./datasets/kc_house_data.csv')

print(f'This is a portion of our dataset:\n{data.iloc[:4,:6]}')

Now we need to pre-process the data and strip it down to only the features we will be using - the price and the area in sqft.

We will then load our stripped-down dataset into x and y variables for our inputs and labels respectively. For now, we'll train the model on 80% of the total examples and leave 20% to validate how our model performs after it's been trained.

In [ ]:
# Strip dataset to columns that represent prices and areas
data_stripped = data.iloc[:,[2,5]]
X = data.iloc[:,[5]] # assign our model inputs as areas
y = data.iloc[:,[2]] # assign our model labels as prices

# 80/20 split for train/validation sets
split_ratio = 0.8
split_index = int(len(data)*split_ratio)

# Assign each split to x,y_train and x,y_val sets & convert to numpy arrays for our model.
x_train = X.iloc[:split_index].values
y_train = y.iloc[:split_index].values
x_val = X.iloc[split_index:].values
y_val = y.iloc[split_index:].values

# Visualize our data splits
print(f'x_train: {x_train}\n')
print(f'y_train: {y_train}\n')
print(f'number of original examples: {len(X)}')
print(f'number of x_train examples: {len(x_train)} | number of x_val examples: {len(x_val)}')
print(f'number of y_train examples: {len(y_train)} | number of y_val examples: {len(y_val)}')

What did we just do?
- Stripped the original dataset down to area features for our inputs to the network (x) and price labels (y). Our model will learn how area affects prices.

- Split our cleaned data into 80% training and 20% validation sets.

- Assigned only the house area column in the dataframe to x_train and x_val, and similarly, only the price column to y_train and y_val.

- Finally converted x_train/x_val and y_train/y_val to numpy arrays with *.values* so our model can ingest them properly.

Before we work on defining our model, as a final preprocessing step, we need to normalize our data that will be going into the model during training.

If we ingest large values for area/prices, our model's computations will be dealing in similarly large values. This can cause numerical instability during training, resulting in extreme and exploding gradients.

Let's normalize our training data with z-score normalization according to the formula:
$$
\text{X}_{std} = \frac{X - \mu}{\sigma}
$$
to center our data around the mean at 0, and scale the spread of our data to a standard deviation of 1.

In [ ]:
print(f'y_train before norm: {y_train}')
print(f'y_val before norm: {y_val}')

# Compute the means
x_mean = np.mean(x_train, axis=0)
y_mean = np.mean(y_train, axis=0)
x_std = np.std(x_train, axis=0)
y_std = np.std(y_train, axis=0)
# Standardize our training data
x_train = (x_train - x_mean) / x_std
y_train = (y_train - y_mean) / y_std

# Standardize our validation data
# NOTE: we use the std and mean of the *training* data to ensure scaling consistency across both sets
# we can't use std or mean of val data for scaling as that leaks info of the val set to the model during training
x_val = (x_val - x_mean) / x_std
y_val = (y_val - y_mean) / y_std

print(f'y_train after norm: {y_train}')
print(f'y_val after norm: {y_val}')

# Model Structure:

We want to define our model structure within a class that we can use easily later. We need to tell the class that, upon initialization of a class instance, create weights and bias variables within which we will eventually store and update our model's parameters.

In [220]:
# Create a class for our model to hold everything
class LinearRegressionModel:
  def __init__(self):
    self.weights = None
    self.bias = None

Now that we have defined how to initialize an instance of our model, we should define how the model will train (also referred to as 'fitting' the data)

In [180]:
# Create a class for our model to hold everything
class LinearRegressionModel:
  def __init__(self):
    self.weights = None
    self.bias = None

# train() will take in:
# - our training data X_train & y_train
# - a parameter 'epochs' for number of training cycles
# - a parameter 'learning_rate' to nudge our weights & biases a small amount each cycle
# - a parameter 'reg' to penalize large weights

  def train(self, x_train, y_train, epochs=100, learning_rate=1e-3, reg=0.01):

    num_train, num_features = x_train.shape # use the dimensions of x_train to get number of training examples/features per example

    self.weights = np.random.rand(num_features, 1)
    self.bias = 0
    train_loss_history = []
    val_loss_history = []

    for epoch in range(epochs):
      # Forward pass - make a prediction
      # we use np.dot() as a fast vectorized approach for when we have multiple features
      y_pred = np.dot(x_train, self.weights) + self.bias

      # Backward pass
      # Compute mean-squared-error loss for the training set (see how wrong the prediction was)
      train_loss = np.mean((y_train - y_pred)**2) + reg * np.sum(self.weights **2)
      train_loss_history.append(train_loss) # save loss each epoch for graphing later

      # Compute the gradients that will update our parameters
      # Average over the number of training examples and add the derivative of the regularization term
      dw = (np.dot(x_train.T, (y_pred - y_train)) / num_train) + (2 * reg * self.weights)
      db = np.sum(y_pred - y_train) / num_train

      # Nudge the old weights & biases by a small amount, scaled by the current gradients
      # these dw,db gradients are the slope of the loss
      # the -= is a *negative* accumulation because we want to descend in the direction of the slope!
      self.weights -= learning_rate * dw
      self.bias -= learning_rate * db

      # Compute MSE loss for our validation set to compare
      y_pred_val = np.dot(x_val, self.weights) + self.bias
      val_loss = np.mean((y_val - y_pred_val)**2) + reg * np.sum(self.weights **2)
      val_loss_history.append(val_loss)

      # Print loss every 10 epochs
      if epoch % 10 == 0:
        print(f'Epoch {epoch}/{epochs} - Train Loss: {train_loss} | Val loss: {val_loss}')

    return train_loss_history, val_loss_history # return these so we can plot the loss history graphs

  # Define a simple prediction function for test-time
  def predict(self, x):
    prediction = np.dot(x, self.weights) + self.bias
    return prediction

Now that our structure and training loop has all been defined, we are ready to hit the big red button and see what it can do!

We'll call the model class and assign it to a variable called *model*.

Then we'll train the model by feeding it our training data (x_train, y_train) with the return values of train() inside the model being stored in *train_loss_history* and *val_loss_history* so we can see how it does!

In [ ]:
# Train for number of epochs (feel free to experiment! more epochs gives the model longer to converge)
epochs = 500
# How much to nudge the parameters by each epoch (smaller lr will take longer to converge)
learning_rate = 1e-2

# Regularization parameter
# if the train/validation sets are very different, try increasing to prevent overfitting!
reg = 0.1

model = LinearRegressionModel()
train_loss_history, val_loss_history = model.train(x_train, y_train, epochs, learning_rate)

In [ ]:
# Plot 2 overlaid curves - train loss and val loss
plt.plot(train_loss_history, label='Training loss')
plt.plot(val_loss_history, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

And there you have it! Our model has converged which is always a good sign!

Though as is evident, our model has a pretty decent sized spread between the train loss and validation loss.

Our linear regression model is only learning a function based on one feature - area. There isn't a huge amount of information it can really glean from just one feature, which is why we converge a bit and then don't go much further - it's learned everything it can about the training data.

Imagine if you knew a house's area in square feet, but also its age, and the number of rooms and the area.. you could definitely get a better idea of its price than just the area!

We need to introduce a few more features in order to add more complexity to the model and hopefully we'll see it glean better insights!

When a linear regression model takes in multiple features, it becomes a ***multiple regression model***.

# Multiple Regression

In [ ]:
# Strip dataset to columns that now represent prices, bedrooms, bathrooms and areas
data_stripped = data.iloc[:,[2,3,4,5]]
X = data.iloc[:,[3,4,5]] # assign our model inputs as [bedrooms, bathrooms, area] for each example
y = data.iloc[:,[2]] # assign our model labels as prices

# 80/15/5 split for train/validation/test sets (we now want a test set to test our model on predictions!)
total_examples = len(data)
train_split_index = int(total_examples * 0.8)
val_split_index = int(total_examples * 0.95)

# Assign each split to x,y_train and x,y_val sets & convert to numpy arrays for our model.
x_train = X.iloc[:split_index].values
y_train = y.iloc[:split_index].values
x_val = X[train_split_index:val_split_index]
y_val = y[train_split_index:val_split_index]
x_test = X[val_split_index:]
y_test = y[val_split_index:]

# Visualize our data splits
print(f'x_test: {x_test}\n')
print(f'y_test: {y_test}\n')
print(f'number of original examples: {len(X)}')
print(f'number of x_train examples: {len(x_train)} | number of x_val examples: {len(x_val)}')
print(f'number of y_train examples: {len(y_train)} | number of y_val examples: {len(y_val)}')
print(f'number of x_test examples: {len(x_test)} | number of y_test examples: {len(y_test)}')

In [184]:
# Compute the means & stds
x_mean = np.mean(x_train, axis=0)
x_std = np.std(x_train, axis=0)
y_mean = np.mean(y_train, axis=0)
y_std = np.std(y_train, axis=0)

# Standardize our training data
x_train = (x_train - x_mean) / x_std
y_train = (y_train - y_mean) / y_std

# Standardize our validation data
x_val = (x_val - x_mean) / x_std
y_val = (y_val - y_mean) / y_std

# Standardize our test data
x_test = (x_test - x_mean) / x_std
y_test = (y_test - y_mean) / y_std

In [ ]:
# Train for number of epochs (feel free to experiment! more epochs gives the model longer to converge)
epochs = 500
# How much to nudge the parameters by each epoch (smaller lr will take longer to converge)
learning_rate = 1e-2

# Regularization parameter - if the train/validation sets are very different, try increasing to prevent overfitting!
reg=0.1

model = LinearRegressionModel()
train_loss_history, val_loss_history = model.train(x_train, y_train, epochs, learning_rate)

In [ ]:
# Plot 2 overlaid curves - train loss and val loss
plt.plot(train_loss_history, label='Training loss')
plt.plot(val_loss_history, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Test our model with the test set!
y_pred_test = model.predict(x_test)
test_error = np.mean((y_test - y_pred_test) **2)
print(f'Test error: {test_error}')

That's about the extent we are going to get into for this notebook.

Our model converged, and both train and val sets showed pretty similar curves so we weren't overfitting. This means the model is generalizing well to data it hasn't seen before!

We could further improve this model by feature scaling our data. Adding polynomial features or other transformations to the training data can significantly help capture more complexity and get the loss even lower!

Hope you had fun and learned something new!